In [1]:
from torchtext import data, datasets
import torch
import numpy  as np
from torch import nn, optim

In [2]:
questions =  data.Field(tokenize = 'spacy', batch_first = True) # uses spacy for tokenization
labels = data.LabelField(dtype = torch.float) # sets as floats so we can caluclate gradients.

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
train_data, _ = datasets.TREC.splits(questions, labels)
train_data, valid_data = train_data.split()
print(len(train_data), len(valid_data))

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


3816 1636


In [4]:
# visualise sample
ith = np.random.randint(len(train_data))
print(train_data.examples[ith].text)
print(train_data.examples[ith].label)

['Who', 'was', 'the', 'famous', 'door', '-', 'to', '-', 'door', 'brush', 'salesman', '?']
HUM


In [5]:
questions.build_vocab(train_data, vectors="glove.6B.200d", unk_init=torch.Tensor.normal_)
labels.build_vocab(train_data)

In [6]:
questions.vocab.vectors

tensor([[ 0.3054, -0.1946, -0.7028,  ...,  0.3361, -1.2243,  1.1565],
        [-1.0522,  0.7952, -1.7471,  ...,  0.9877,  1.5790,  1.4732],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.2698, -0.3513,  0.0998,  ...,  0.1792,  0.0125, -0.0725],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570],
        [-0.3643, -0.6794,  0.3678,  ..., -0.0382,  1.1400, -0.1825]])

In [7]:
# create data iterators
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = 64,
    device = device
)


/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [12]:
class CNN(nn.Module):
    def __init__(self,
                 vocab_size, 
                 embedding_dim, 
                 n_filters, 
                 filter_sizes, 
                 output_dim, 
                 dropout, 
                 pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                      embedding_dim=embedding_dim, 
                                      padding_idx=pad_idx) # padding_index ->index of embeddign used to pad to same length, define later manually
        self.convs = nn.ModuleList( # define a list of convs layers
            [nn.Conv2d(in_channels=1, # input text does not have channels
                       out_channels=n_filters, # how many conv filters with same shape to train (same for all filter_sizes) 
                       kernel_size=(fs, embedding_dim)
                      ) 
             for fs in filter_sizes]
        )
        
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout= nn.Dropout(dropout)
        
        
    def forward(self, text):
        emb = self.embedding(text).unsqueeze(1)
        conved = [F.relu(conv2d(emb)).squeeze(3) for conv2d in self.convs]
        pooled = [F.mac_pool1d(c , c.shape[2]).squeeze(2) for c in conved]
        concat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(concat)
    

In [13]:
input_dimension = len(questions.vocab)
output_dimension = 6
embedding_dimension = 200
pad_index = questions.vocab.stoi[questions.pad_token]
number_of_filters = 100
filter_sizes = [2, 3, 4]
dropout_pc = 0.5

model = CNN(vocab_size=input_dimension, 
                 embedding_dim = embedding_dimension, 
                 n_filters = number_of_filters, 
                 filter_sizes = filter_sizes, 
                 output_dim = output_dimension, 
                 dropout=dropout_pc, 
                 pad_idx=pad_index)

In [16]:
# load glove as embeddings  for tokens
glove_embeddings = questions.vocab.vectors
model.embedding.weight.data.copy_(glove_embeddings)

tensor([[ 0.3054, -0.1946, -0.7028,  ...,  0.3361, -1.2243,  1.1565],
        [-1.0522,  0.7952, -1.7471,  ...,  0.9877,  1.5790,  1.4732],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.2698, -0.3513,  0.0998,  ...,  0.1792,  0.0125, -0.0725],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570],
        [-0.3643, -0.6794,  0.3678,  ..., -0.0382,  1.1400, -0.1825]])

In [19]:
# handels unknown and paddings
unknown_index = questions.vocab.stoi[questions.unk_token]
model.embedding.weight.data[unknown_index] = torch.zeros(embedding_dimension)
model.embedding.weight.data[pad_index] = torch.zeros(embedding_dimension)


In [20]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)
model = model.to(device)